### 01-school-closure-clean.ipynb
* **Script**: 01-school-closure-clean.ipynb
* **Purpose**: Import raw NCES CCD and EdFacts data, clean, and engineer features.
* **Input**: NCES CCD School-Level Data from 2009-10 (download [here](https://nces.ed.gov/ccd/pubschuniv.asp)), EdFacts Math/ELA Percent Proficiency Data for 2009-10 (download [here](http://www2.ed.gov/about/inits/ed/edfacts/data-files/index.html)).
* **Output**: data/school_closure_clean.pkl

In [8]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

#### Import 2009-10 NCES School-Level CCD File. 
(Data not saved to GH - You may have to adjust the file path depending on where you save and unzip the files)

In [9]:
variables=["NCESSCH","LEAID","LEANM09","SCHNAM09","LSTATE09","LEVEL09","TYPE09",\
           "STATUS09","ULOCAL09","FTE09","TITLEI09","STITLI09","MAGNET09","CHARTR09",\
           "SHARED09","TOTFRL09","MEMBER09","WHITE09","TOTETH09"]

school=pd.read_table("data/ccd/2009-10/sc092a.txt")[variables]

#Remove "09" from variable name suffixes
vardict = {}
for variable in variables:
    if variable[-2:]=="09":
        vardict[variable]=variable[:-2]
school.rename(columns = vardict, inplace=True)
        
##Subset to only regular schools operational in 2009-10
school=school[(school.TYPE==1) & (school.STATUS.isin([1,3,4,5,8]))]
print school.shape

(90969, 19)


#### Engineer Features
#### Pool states into geographic regions (based on [U.S. Census Definitions](http://www.census.gov/econ/census/help/geography/regions_and_divisions.html))

In [10]:
def pool(df, pooled, unpooled, values):
    df[pooled]=0
    df.ix[df[unpooled].isnull(), pooled]=np.nan
    df.ix[df[unpooled].isin(values), pooled]=1

pool(school, "NEW_ENGLAND","LSTATE",["CT","ME","MA","NH","RI","VT"])
pool(school, "MID_ATLANTIC","LSTATE",["NJ","NY","PA"])
pool(school, "EAST_NORTH_CENTL","LSTATE",["IL","IN","MI","OH","WI","AE"])
pool(school, "WEST_NORTH_CENTL","LSTATE",["IA","KS","MN","MO","NE","ND","SD"])
pool(school, "SOUTH_ATLANTIC", "LSTATE",["DE","DC","FL","GA","MD","NC","SC","VA","WV","VI","PR"])
pool(school, "EAST_SOUTH_CENTL", "LSTATE",["AL","KY","MS","TN"])
pool(school, "WEST_SOUTH_CENTL", "LSTATE",["AR","LA","OK","TX"])
pool(school, "MOUNTAIN", "LSTATE",["AZ","CA","CO","ID","MT","NV","NM","UT","WY"])    
pool(school, "PACIFIC", "LSTATE", ["AK","HI","OR","WA","AP","AS","MP","GU"])

print school[["NEW_ENGLAND","MID_ATLANTIC","EAST_NORTH_CENTL","WEST_NORTH_CENTL",\
                     "SOUTH_ATLANTIC","EAST_SOUTH_CENTL","WEST_SOUTH_CENTL",\
             "MOUNTAIN","PACIFIC"]].sum(axis=1).value_counts(dropna=False)

1    90969
dtype: int64


#### Pool urbanacities into general categories (Urban, Suburb, Town, Rural)

In [11]:
school["ULOCAL"]=pd.to_numeric(school.ULOCAL, errors="coerce")

pool(school, "CITY",  "ULOCAL",[11,12,13])
pool(school, "SUBURB","ULOCAL",[21,22,23])
pool(school, "TOWN",  "ULOCAL",[31,32,33])
pool(school, "RURAL", "ULOCAL",[41,42,43])

print pd.crosstab(school.ULOCAL, school.CITY)
print pd.crosstab(school.ULOCAL, school.SUBURB)
print pd.crosstab(school.ULOCAL, school.TOWN)
print pd.crosstab(school.ULOCAL, school.RURAL)

CITY        0      1
ULOCAL              
11          0  12026
12          0   4977
13          0   5984
21      21103      0
22       2670      0
23       1742      0
31       1775      0
32       6151      0
33       4301      0
41      12116      0
42      10964      0
43       6695      0
SUBURB      0      1
ULOCAL              
11      12026      0
12       4977      0
13       5984      0
21          0  21103
22          0   2670
23          0   1742
31       1775      0
32       6151      0
33       4301      0
41      12116      0
42      10964      0
43       6695      0
TOWN        0     1
ULOCAL             
11      12026     0
12       4977     0
13       5984     0
21      21103     0
22       2670     0
23       1742     0
31          0  1775
32          0  6151
33          0  4301
41      12116     0
42      10964     0
43       6695     0
RURAL       0      1
ULOCAL              
11      12026      0
12       4977      0
13       5984      0
21      21103      0
22    

#### Clean Magnet, Charter and Shared-Time School Flags

In [12]:
print school.MAGNET.value_counts(dropna=False)
print school.CHARTR.value_counts(dropna=False)
print school.SHARED.value_counts(dropna=False)

school["MAGNET"]=pd.to_numeric(school.MAGNET, errors="coerce")
school["CHARTR"]=pd.to_numeric(school.CHARTR, errors="coerce")
school["SHARED"]=pd.to_numeric(school.SHARED, errors="coerce")

school.ix[school.MAGNET !=1, "MAGNET"]=0
school.ix[school.CHARTR !=1, "CHARTR"]=0
school.ix[school.SHARED !=1, "SHARED"]=0

print school.MAGNET.value_counts(dropna=False)
print school.CHARTR.value_counts(dropna=False)
print school.SHARED.value_counts(dropna=False)

2    39674
N    29948
M    11713
2     7476
1     1931
1      227
Name: MAGNET, dtype: int64
2    62245
2    13209
N    11051
1     3734
1      698
M       32
Name: CHARTR, dtype: int64
2    61656
M    17407
2    11145
1      754
1        7
Name: SHARED, dtype: int64
0    88811
1     2158
Name: MAGNET, dtype: int64
0    86537
1     4432
Name: CHARTR, dtype: int64
0    90208
1      761
Name: SHARED, dtype: int64


#### Break school level variables into indidvidual 0/1 flags

In [13]:
#School Level (ELEMENTARY, MIDDLE, HIGH, OTHER)
print school.LEVEL.value_counts(dropna=False)

school["ELEM"]=0
school["MIDDLE"]=0
school["HIGH"]=0
school["OTHER"]=0

school.ix[school.LEVEL=="1", "ELEM"]=1
school.ix[school.LEVEL=="2", "MIDDLE"]=1
school.ix[school.LEVEL=="3", "HIGH"]=1
school.ix[school.LEVEL.isin(["4","N"]),"OTHER"]=1

print pd.crosstab(school.LEVEL, school.ELEM)
print pd.crosstab(school.LEVEL, school.MIDDLE)
print pd.crosstab(school.LEVEL, school.HIGH)
print pd.crosstab(school.LEVEL, school.OTHER)

1    53701
2    16790
3    16465
4     3745
N      268
Name: LEVEL, dtype: int64
ELEM       0      1
LEVEL              
1          0  53701
2      16790      0
3      16465      0
4       3745      0
N        268      0
MIDDLE      0      1
LEVEL               
1       53701      0
2           0  16790
3       16465      0
4        3745      0
N         268      0
HIGH       0      1
LEVEL              
1      53701      0
2      16790      0
3          0  16465
4       3745      0
N        268      0
OTHER      0     1
LEVEL             
1      53701     0
2      16790     0
3      16465     0
4          0  3745
N          0   268


#### Create Title I Flags (All Students, Some Students, No Students)

In [14]:
pd.crosstab(school.TITLEI, school.STITLI)

STITLI,1,2,M,N
TITLEI,,,,
1,44275,19738,0,0
2,0,0,0,25528
M,0,0,637,0
N,0,0,0,791


In [15]:
school["T1_ALL"]=0
school["T1_SOME"]=0
school["T1_NONE"]=0

school.ix[school.TITLEI=="M", "T1_ALL"]=np.nan
school.ix[school.TITLEI=="M", "T1_SOME"]=np.nan
school.ix[school.TITLEI=="M", "T1_NONE"]=np.nan

school.ix[(school.TITLEI=="1") & (school.STITLI=="1"), "T1_ALL"]=1
school.ix[(school.TITLEI=="1") & (school.STITLI=="2"), "T1_SOME"]=1
school.ix[(school.TITLEI.isin(["2","N"])), "T1_NONE"]=1

print school.T1_ALL.value_counts(dropna=False)
print school.T1_SOME.value_counts(dropna=False)
print school.T1_NONE.value_counts(dropna=False)

 0     46057
 1     44275
NaN      637
Name: T1_ALL, dtype: int64
 0     70594
 1     19738
NaN      637
Name: T1_SOME, dtype: int64
 0     64013
 1     26319
NaN      637
Name: T1_NONE, dtype: int64


#### Define a function to trim top and bottom 1st pecentiles (e.g outliers) from numeric features

In [25]:
def trim_outliers(df, variable):
    p01=school[variable].quantile(q=0.01)
    p99=school[variable].quantile(q=0.99)
    
    school.ix[(school[variable]<p01) | (school[variable]>p99), variable]=np.nan

#### Calculate School Percent Minority Students

In [26]:
school.ix[school.TOTETH<0, "TOTETH"]=np.nan
school.ix[school.WHITE<0, "WHITE"]=np.nan

trim_outliers(school, "TOTETH") #clean outliers - drop top and bottom 1st percentiles
trim_outliers(school, "WHITE") #clean outliers - drop top and bottom 1st percentiles

school["PCT_MINORITY"]=((school.TOTETH-school.WHITE)/school.TOTETH)
school[["SCHNAM","WHITE","TOTETH","PCT_MINORITY"]].head(5)

,SCHNAM,WHITE,TOTETH,PCT_MINORITY
6,ALABAMA YOUTH SERVICES,NaN,NaN,NaN
7,ALA AVENUE MIDDLE SCH,373,526,0.290875
8,ALBERTVILLE HIGH SCH,789,1045,0.244976
9,EVANS ELEM SCH,395,600,0.341667
10,ALBERTVILLE ELEM SCH,384,644,0.403727


#### Calculate Percent Free/ Reduced Price Lunch Students

In [27]:
#Percent Free/ Reduced Price Lunch
school.ix[school.MEMBER<0, "MEMBER"]=np.nan
school.ix[school.TOTFRL<0, "TOTFRL"]=np.nan

trim_outliers(school, "MEMBER")
trim_outliers(school, "TOTFRL")

school["PCT_FRL"]=(school.TOTFRL/ school.MEMBER)
school[["SCHNAM","MEMBER","TOTFRL","PCT_FRL"]].head(5)

,SCHNAM,MEMBER,TOTFRL,PCT_FRL
6,ALABAMA YOUTH SERVICES,NaN,NaN,NaN
7,ALA AVENUE MIDDLE SCH,528,320,0.606061
8,ALBERTVILLE HIGH SCH,1052,471,0.447719
9,EVANS ELEM SCH,600,394,0.656667
10,ALBERTVILLE ELEM SCH,644,460,0.714286


In [28]:
school[["TOTETH","WHITE","MEMBER","TOTFRL","PCT_MINORITY","PCT_FRL"]].describe()

,TOTETH,WHITE,MEMBER,TOTFRL,PCT_MINORITY,PCT_FRL
count,85055.000000,87643.000000,85079.000000,84432.000000,83769.000000,81173.000000
mean,496.739745,253.838561,501.916830,230.582563,0.429539,0.501508
std,296.669892,219.281396,299.107899,178.454440,0.349034,0.271451
min,46.000000,0.000000,46.000000,0.000000,0.000000,0.000000
25%,288.000000,65.000000,291.000000,88.000000,0.103371,0.288043
50%,446.000000,215.000000,452.000000,190.000000,0.336774,0.494828
75%,641.000000,384.000000,648.000000,335.000000,0.766917,0.718708
max,1659.000000,1018.000000,1674.000000,811.000000,1.000000,0.998440


#### Calculate Student-Teacher Ratio

In [29]:
#Student-Teacher Ratio
school.ix[school.FTE<=0, "FTE"]=np.nan
trim_outliers(school, "FTE")

school["ST_RATIO"]=school.MEMBER/ school.FTE
trim_outliers(school, "ST_RATIO")
print school["ST_RATIO"].describe()

count    81926.000000
mean        15.753743
std          3.628499
min          7.584526
25%         13.238095
50%         15.364765
75%         17.935484
max         27.011309
Name: ST_RATIO, dtype: float64


#### Calculate By-State Percentile Performance on Statewide Standardized Tests
#### Read in EdFacts Math and ELA Percent Proficient data for 2009-10 & merge together
(These data can be downloaded [here](http://www2.ed.gov/about/inits/ed/edfacts/data-files/index.html)).

In [31]:
mathvars=["STNAM","NCESSCH","ALL_MTH00pctprof_0910"]
elavars=["STNAM","NCESSCH","ALL_RLA00pctprof_0910"]

math=pd.read_csv("data/EDfacts/2009-10/math-achievement-sch-sy2009-10.csv")[mathvars]
ela=pd.read_csv("data/EDfacts/2009-10/rla-achievement-sch-sy2009-10.csv")[elavars]

asmt=pd.merge(math,ela,on=["STNAM","NCESSCH"])
asmt.rename(columns={"ALL_MTH00pctprof_0910":"MATH_PROF","ALL_RLA00pctprof_0910":"ELA_PROF"}, inplace=True)

print math.shape, ela.shape, asmt.shape
print asmt.columns

(91205, 3) (91142, 3) (91086, 4)
Index([u'STNAM', u'NCESSCH', u'MATH_PROF', u'ELA_PROF'], dtype='object')


#### Clean Proficiency Rate Data

In [32]:
def clean_score(raw):
    raw=str(raw)
    
    if raw in ["PS","n/a","GE50","GE40","GE30","LT50","LT40","LT30"]:
        clean=np.nan
        
    elif raw[:2] in ["GE","GT","LE","LT"]: 
        clean=float(raw[2:])

    else:
        split=str(raw).split("-")
    
        if len(split)==1:
            clean=float(split[0])
    
        elif len(split)==2:
            clean=np.mean([float(split[0]),float(split[1])])
        
        else:
            print "ERROR CHECK RANGE: "+split
    
    return clean

asmt["MATH_PROF_CLN"]=asmt.MATH_PROF.apply(clean_score)
asmt["ELA_PROF_CLN"]=asmt.ELA_PROF.apply(clean_score)

print asmt[asmt.MATH_PROF_CLN.isnull()]["MATH_PROF"].value_counts(dropna=False)
print asmt[asmt.ELA_PROF_CLN.isnull()]["ELA_PROF"].value_counts(dropna=False)

PS      1392
LT50    1360
GE50     780
n/a      323
Name: MATH_PROF, dtype: int64
PS      1401
GE50    1118
LT50    1003
n/a      331
NaN        1
Name: ELA_PROF, dtype: int64


#### Calculate By-State Percentile Ranks in Math and ELA

In [33]:
ranks=pd.DataFrame()

for state in asmt.STNAM.unique():
    state_ranks=asmt[asmt.STNAM==state].copy()
    
    state_ranks["MATH_RANK"]=asmt.MATH_PROF_CLN.rank(pct=True)
    state_ranks["ELA_RANK"]=asmt.ELA_PROF_CLN.rank(pct=True)
    
    ranks=pd.concat([ranks,state_ranks], axis=0)

ranks["in_ranks"]=1 #Flag schools that are in the EdFacts Data

print asmt.shape
print ranks.shape
print ranks.describe()

(91086, 6)
(91086, 9)
            NCESSCH  MATH_PROF_CLN  ELA_PROF_CLN     MATH_RANK      ELA_RANK  \
count  9.108600e+04   87231.000000  87232.000000  87231.000000  87232.000000   
mean   2.899319e+11      69.167251     70.629838      0.500006      0.500006   
std    1.676538e+11      21.631216     19.878451      0.288579      0.288566   
min    1.000050e+10       1.000000      3.000000      0.000722      0.000017   
25%    1.302940e+11      56.000000     57.000000      0.248816      0.238559   
50%    2.905680e+11      74.500000     76.000000      0.499249      0.504379   
75%    4.212210e+11      87.000000     87.000000      0.765611      0.762897   
max    7.200030e+11      99.000000     99.000000      0.996687      0.997128   

       in_ranks  
count     91086  
mean          1  
std           0  
min           1  
25%           1  
50%           1  
75%           1  
max           1  


#### Match State Assmt Data to NCES CCD Data

In [34]:
merged=pd.merge(school,ranks,on="NCESSCH",how="left")
merged.ix[merged.in_ranks.isnull(), "in_ranks"]=0 #if record is not in EdFacts data, set in_ranks=0

print merged.in_ranks.value_counts()
print school.shape, merged.shape

merged["intercept"]=1 #add intercept term for modelling

1    85870
0     5099
Name: in_ranks, dtype: int64
(90969, 42) (90969, 50)


#### Engineer Label (closed within five years of 2009-10 (base year))
Use data from the 2010-11 - 2014-15 NCES-CCD files. Again, these can be downloaded [here](https://nces.ed.gov/ccd/pubschuniv.asp). Again, you may have to adjust file paths depending on where data were saved.

In [35]:
def flag_closure(yr, data, closed):
    df=pd.read_table("data/ccd/"+yr+"/"+data)[["NCESSCH","STATUS"]]
    
    df[closed]=0
    df.ix[df["STATUS"].isin([2,6]), closed]=1
  
    return df[["NCESSCH",closed]]
    
closed10=flag_closure("2010-11", "sc102a.txt", "CLOSED10")
closed11=flag_closure("2011-12", "sc111a_supp.txt", "CLOSED11")
closed12=flag_closure("2012-13", "sc122a.txt", "CLOSED12")
closed13=flag_closure("2013-14", "sc132a.txt", "CLOSED13")
closed14=flag_closure("2014-15", "Sch14pre.txt", "CLOSED14")

merged=pd.merge(merged, closed10, on="NCESSCH", how="left")
merged=pd.merge(merged, closed11, on="NCESSCH", how="left")
merged=pd.merge(merged, closed12, on="NCESSCH", how="left")
merged=pd.merge(merged, closed13, on="NCESSCH", how="left")
merged=pd.merge(merged, closed14, on="NCESSCH", how="left")

print merged.shape
print merged.columns

/Users/willhuguenin/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:3006: DtypeWarning: Columns (5,6,9,18,22,25,27,53,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/willhuguenin/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:3006: DtypeWarning: Columns (5,6,9,22,23,26,28,29,67,68,69) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/willhuguenin/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:3006: DtypeWarning: Columns (5,6,9,22,23,26,28,54,55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/willhuguenin/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:3006: DtypeWarning: Columns (9,22) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):

(90969, 56)
Index([u'NCESSCH', u'LEAID', u'LEANM', u'SCHNAM', u'LSTATE', u'LEVEL', u'TYPE',
       u'STATUS', u'ULOCAL', u'FTE', u'TITLEI', u'STITLI', u'MAGNET',
       u'CHARTR', u'SHARED', u'TOTFRL', u'MEMBER', u'WHITE', u'TOTETH',
       u'NEW_ENGLAND', u'MID_ATLANTIC', u'EAST_NORTH_CENTL',
       u'WEST_NORTH_CENTL', u'SOUTH_ATLANTIC', u'EAST_SOUTH_CENTL',
       u'WEST_SOUTH_CENTL', u'MOUNTAIN', u'PACIFIC', u'CITY', u'SUBURB',
       u'TOWN', u'RURAL', u'ELEM', u'MIDDLE', u'HIGH', u'OTHER', u'T1_ALL',
       u'T1_SOME', u'T1_NONE', u'PCT_MINORITY', u'PCT_FRL', u'ST_RATIO',
       u'STNAM', u'MATH_PROF', u'ELA_PROF', u'MATH_PROF_CLN', u'ELA_PROF_CLN',
       u'MATH_RANK', u'ELA_RANK', u'in_ranks', u'intercept', u'CLOSED10',
       u'CLOSED11', u'CLOSED12', u'CLOSED13', u'CLOSED14'],
      dtype='object')


#### If a school closed during any year during the five year period, flag as closed. Otherwise, flag as not closed

In [36]:
merged["CLOSED"]=merged[["CLOSED10","CLOSED11","CLOSED12","CLOSED13","CLOSED14"]].max(axis=1)
print merged.CLOSED.value_counts(dropna=False)

 0     85046
 1      5667
NaN      256
Name: CLOSED, dtype: int64


#### Remove data with missing label (only ~2.8%). Pickle dataframe for modelling.

In [37]:
merged=merged[merged.CLOSED.notnull()]
merged.to_pickle("data/school_closure_clean.pkl")